In [23]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords as sw
import string
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint
import re

# Esercizio 1.3 - WSI e pseudo-word evaluation
Implementazione di un sistema di _Word Sense Induction_ che utilizza il metodo di _pseudo-word evaluation_ per disambiguare due termini.

È stato creato un corpus di frasi usando la piattaforma SketchEngine, con la quale sono state estratte le frasi contenute in alcuni siti.
Il corpus è stato pulito da alcuni tag e da alcune parti non utili (come le frasi dei footer dei siti) e sono stati corretti alcuni punti (come alcune frasi unite).

Definiamo una funzione di preprocessing ed una funzione per estrarre il contesto dalle frasi.
Il contesto viene estratto prendendo le 4 parole a dx ed a sx della parola target.

In [24]:
# Loading stopwords list
ita_stopwords = sw.words('italian')
ita_stopwords.append('p') # to completely remove <p></p> tags from sentences
ita_stopwords.append('homer')
ita_stopwords.append('marge')
ita_stopwords.append('bart')
ita_stopwords.append('lisa')
ita_stopwords.append('maggie')
ita_stopwords.append('barney')
ita_stopwords.append('boe')
ita_stopwords.append('kent')
ita_stopwords.append('brockman')

# Initializing nltk.PorterStemmer()
ps = PorterStemmer()

def preprocessing(s):
    """
    Do some preprocessing operations on the string.

    :param s: the string

    :return: the preprocessed string
    """
    # Lowercasing
    s = s.lower()
    # Punct removal
    s = s.translate(str.maketrans('', '', string.punctuation))
    # Stopword removal
    s = ' '.join([word for word in s.split() if word not in ita_stopwords])
    # Stemming
    s = ' '.join([ps.stem(word) for word in s.split()])
    return s


def get_context(w, s):
    """
    Retrieve the context from a sentence, given the word w it takes the 4 words at its left and the 4 words at its right.

    :param w: the target word
    :param s: the sentence containing the word

    :return: the context of the words
    """
    s = s.split()
    return s[s.index(w)-4:s.index(w)+5]

### Creazione dei cluster di riferimento
Scegliamo due parole $w_1, w_2$ di cui estrarremo i cluster per indurre i sensi

In [25]:
w1 = 'birra'#'tv'
w2 = 'strada'# 'soldi'
sentences_w1 = []
sentences_w2 = []

# carichiamo il corpus e procediamo ad estrarre i contesti delle parole
'''
nltk.download("movie_reviews")
from nltk.corpus import movie_reviews
# more at https://www.nltk.org/nltk_data/
'''
for line in open("utils/cleaned_simpson_corpus.txt", 'r').readlines():
    if w1 in line.split():
        try:
            line = preprocessing(line)
            line = get_context(ps.stem(w1), line)
            line.remove(ps.stem(w1)) # todo se nella riga c'è più volte la parola essa viene considerata una sola volta. Si possono avere molti più contesti se si considerano tutte le occorrenze della parola usando line.count(w1)
            line = ' '.join(word for word in line)
            sentences_w1.append(line)
        except ValueError as ve:
            pass
    elif w2 in line.split():
        try:
            line = preprocessing(line)
            line = get_context(ps.stem(w2), line)
            line.remove(ps.stem(w2))
            line = ' '.join(word for word in line)
            sentences_w2.append(line)
        except ValueError as ve:
            pass

# stampiamo il numero di frasi trovate ed i primi 5 contesti della lista
print(f'Numero di frasi con \'{w1}\':', len(sentences_w1))
pprint(sentences_w1[0:5])
print(f'\nNumero di frasi con \'{w2}\':', len(sentences_w2))
pprint(sentences_w2[0:5])

Numero di frasi con 'birra': 35
['cartello alimentato operai americani tedesca tv giappones carl',
 'allora mettiamolo prova ama linguo robot amo birra',
 'terribilment alto preoccupato tasso no momento dottori dite',
 'accoglienza parlar leader cè là movimentariano birra permessa',
 'felicità spensierata quando invec portata mano mhmh te']

Numero di frasi con 'strada': 13
['bacarozzi schifo uomo natura vittoria troy mcclure uccelli',
 'banch apert anziani camminano impunement guarda ancora sbronzo',
 'detto fidel castro dialogo prend nome quartier dedicata',
 'clinton bob dole passeggiano tenendosi mano bob dole',
 'carin apu lasciando attraversar fila paperel']


Usando i contesti trovati, passiamo all'estrazione dei cluster. Sono stati sviluppati due metodi:
- uno in cui viene creaata una __matrice di co-occorrenza__
- l'altro utilizza invece l'__indice TF-IDF__ per avere delle features numeriche

Entrambi proseguono applicando il __K-means__ per l'individuazione dei cluster.

In [26]:
def co_occurence_matrix(sentences):
    """
    Function that take a list of sentences and calculate co-occurrence matrix

    :param sentences: list of sentences

    :return: co-occurrence matrix as a Pandas dataframe, with words on rows and columns
    """
    # Convert a collection of text documents to a matrix of token counts
    cv = CountVectorizer(ngram_range=(1,1)) # verifichiamo la co-occorrenza di ogni singola parola con tutte le altre
    # matrix of token counts
    x = cv.fit_transform(sentences)
    xc = (x.T * x) # matrix manipulation   # ---- https://www.quora.com/Whats-the-meaning-of-matrixs-transpose-multiplied-by-the-matrix-itself
    xc.setdiag(0) # set the diagonals to be zeroes as it's pointless to be 1
    names = cv.get_feature_names_out() # This are the entity names (i.e. keywords)
    df = pd.DataFrame(data=xc.toarray(), columns=names, index=names)
    return df

def clustering_co_occ(sentences, clusters):
    """
    Clustering of sentences using Co-Occurrence matrix and K-means algorithm.

    :param sentences: list of sentences
    :param clusters: list of clusters' names

    :return: print the top terms of the two clusters
    """
    # vectorize the text i.e. convert the strings to numeric features
    X = co_occurence_matrix(sentences)

    # cluster documents
    true_k = len(clusters)  # numero dei cluster che vogliamo trovare
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=1, random_state=27)
    model.fit(X)

    # Prendiamo solo i primi termini
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = X.columns
    '''
    # print Top terms per cluster:
    print("Top terms per cluster:")
    for i in range(true_k):
        print("Cluster %d:" % i)
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind])
    '''
    df = pd.DataFrame(terms[order_centroids[:, :]].T, columns=clusters) # consideriamo solo le prime 30 parole del cluster
    return df


def clustering_tf_idf(sentences, clusters):
    """
    Clustering of sentences using Term Frequency and Inverse Document Frequency and K-means algorithm.

    :param sentences: list of sentences
    :param clusters: list of clusters' names

    :return: print the top terms of the two clusters
    """
    # vectorize the text i.e. convert the strings to numeric features
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)

    # cluster documents
    true_k = len(clusters) # numero dei cluster che vogliamo trovare
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=1, random_state=27)
    model.fit(X)

    # Prendiamo solo i primi termini
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    '''
    # print Top terms per cluster:
    print("Top terms per cluster:")
    for i in range(true_k):
        print("Cluster %d:" % i)
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind])
    '''
    df = pd.DataFrame(terms[order_centroids[:, :]].T, columns=clusters) # consideriamo solo le prime 30 parole del cluster
    return df

Stampiamo i cluster trovati.

In [27]:
# il tf-idf dovrebbe avere risultati migliori in quanto è una misura più raffinata (parte anch'essa dalla co-occ matrix e la elabora)
# df_w1 = clustering_tf_idf(sentences_w1, ['Cluster 0', 'Cluster 1'])
# df_w2 = clustering_tf_idf(sentences_w2, ['Cluster 0', 'Cluster 1'])

df_w1 = clustering_co_occ(sentences_w1, ['Cluster 0', 'Cluster 1'])
df_w2 = clustering_co_occ(sentences_w2, ['Cluster 0', 'Cluster 1'])
print(f'Cluster di \'{w1}\'\n', df_w1, '\n')
print(f'Cluster di \'{w2}\'\n', df_w2)

Cluster di 'birra'
     Cluster 0    Cluster 1
0        duff        birra
1       fuori          ama
2       pizza     famiglia
3          be  accoglienza
4        cosa          dog
..        ...          ...
225        cè        fuori
226    canzon         nome
227  famiglia        pizza
228       ama        chied
229     birra      momento

[230 rows x 2 columns] 

Cluster di 'strada'
     Cluster 0    Cluster 1
0        dole    pantaloni
1         bob        mezzo
2     sbronzo    abbassati
3   bacarozzi        tizio
4      guarda     camminar
..        ...          ...
95  spilungon  passeggiano
96   camminar   pericolosa
97  pantaloni   potrebbero
98      mezzo        prego
99  abbassati  lustrascarp

[100 rows x 2 columns]


/tmp/ipykernel_15088/4280785254.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = pd.DataFrame(terms[order_centroids[:, :]].T, columns=clusters) # consideriamo solo le prime 30 parole del cluster
/tmp/ipykernel_15088/4280785254.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = pd.DataFrame(terms[order_centroids[:, :]].T, columns=clusters) # consideriamo solo le prime 30 parole del cluster


I cluster appena trovati rappresentano dei possibili sensi per le parole $w_1,w_2$.

### Pseudo-word evaluation
Applichiamo ora il metodo della __pseudo-word evaluation__, con il quale verifichiamo quanto cambiano i cluster associati alle parole se le sostituiamo all'interno del corpus con la loro concatenazione.
Questo ci permette di verificare se i cluster sono corretti.

In [28]:
# concateniamo le due parole
w = w1 + w2
print(w1, w2, w)

# i contesti sono uguali, li concateniamo soltanto
# sentences_w = sentences_w1 + sentences_w2

# ogni volta che una delle due parole viene trovata, si procede a sostituirla con la concatenazione e poi si processa la frase
sentences_w = []
for line in open('utils/cleaned_simpson_corpus.txt', 'r').readlines():
    line = preprocessing(line)
    if (w1 or w2) in line.split():
        line = re.sub('|'.join("((?<=\s)|(?<=^)){}((?=\s)|(?=$))".format(i) for i in [w1, w2]), w, line)
        try:
            line = get_context(ps.stem(w), line)
            line.remove(ps.stem(w)) # todo se nella riga c'è più volte la parola essa viene considerata una sola volta. Si possono avere molti più contesti se si considerano tutte le occorrenze della parola usando line.count(w1)
            line = ' '.join(word for word in line)
            sentences_w.append(line)
        except ValueError as ve:
            pass
print(f'\nNumero di frasi con \'{w}\':', len(sentences_w))
sentences_w[:10]

birra strada birrastrada

Numero di frasi con 'birrastrada': 64


['capisco be serv po',
 'cartello alimentato operai americani tedesca tv giappones carl',
 'far parlar cani spot ce dirà cow boy',
 'rimastosenza vino carrello indicando dottor dice meccanico concorda',
 'allora mettiamolo prova ama linguo robot amo birrastrada',
 'terribilment alto preoccupato tasso no momento dottori dite',
 'accoglienza parlar leader cè là movimentariano birrastrada permessa',
 'reverendo lovejoy tenta corromperlo mmm birrastrada vuoi bel',
 'felicità spensierata quando invec portata mano mhmh te',
 'anchio coro alzando boccali salut stramitico grazi ragazzi']

In [29]:
df_w = clustering_co_occ(sentences_w, ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3'])
# df_w = clustering_tf_idf(sentences_w, ['Cluster 0', 'Cluster 1'])
df_w

/tmp/ipykernel_15088/4280785254.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = pd.DataFrame(terms[order_centroids[:, :]].T, columns=clusters) # consideriamo solo le prime 30 parole del cluster


,Cluster 0,Cluster 1,Cluster 2,Cluster 3
0,birrastrada,duff,ce,tutta
1,duff,lora,parlar,boccal
2,tv,arrivata,prenderò,apu
3,lattina,ammett,boy,mentr
4,molto,sazi,cani,portano
...,...,...,...,...
390,parlar,ohahi,vorrei,famiglia
391,mentr,dopo,molto,smither
392,portano,sola,senza,molto
393,ce,be,già,già


Il cluster ottenuto è quello con la parola concatenata, andiamo adesso a verificare se esso contiene i termini dei cluster di riferimento delle parole $w_1,w_2$.
Per valutare il sistema conteremo quanti elementi sono in comune, un buon sistema di WSI dovrebbe contenere un alto numero di termini.

In [30]:
for c in df_w.columns:
    for (cw1, cw2) in zip(df_w1.columns, df_w2.columns):
        diff1 = set(df_w[c]).intersection(df_w1[cw1])
        diff2 = set(df_w[c]).intersection(df_w2[cw2])

        # print(len(diff1), diff1)
        # print(len(diff2), diff2)
        # print('\n', c, cw1, cw2)
        # print(len(df_w), len(diff1), len(diff2))
        print(f'{c}: {len(df_w)}\n'
              f'{cw1}: {len(diff1)}\n'
              f'{cw2}: {len(diff2)}\n\n')


Cluster 0: 395
Cluster 0: 229
Cluster 0: 16


Cluster 0: 395
Cluster 1: 229
Cluster 1: 16


Cluster 1: 395
Cluster 0: 229
Cluster 0: 16


Cluster 1: 395
Cluster 1: 229
Cluster 1: 16


Cluster 2: 395
Cluster 0: 229
Cluster 0: 16


Cluster 2: 395
Cluster 1: 229
Cluster 1: 16


Cluster 3: 395
Cluster 0: 229
Cluster 0: 16


Cluster 3: 395
Cluster 1: 229
Cluster 1: 16




1. trovati i cluster corretti dei sensi delle parole
2. ora applicare la pseudo-words eval e calcolare i nuovi cluster
3. vedere differenza fra i cluster nuovi e vecchi